In [536]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import time

In [537]:
html_doc = requests.get('https://www.foxnews.com/politics')
soup = BeautifulSoup(html_doc.text, 'html.parser')

In [538]:
chrome_path = "./chromedriver"
options = webdriver.ChromeOptions()
options.add_argument('--load-extension=./adblock4Sellenium/3.34.0_0')
#options.add_experimental_option( "prefs",{'profile.managed_default_content_settings.javascript': 2})
#options.add_argument("--user-agent=" + 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36')
driver = webdriver.Chrome(chrome_path, options=options)

In [539]:
def filterWords(words):
    newWords = []
    for word in words:
        word = word.lower()
        word = word.replace("!", "")
        word = word.replace("?", "")
        word = word.replace(":", "")
        word = word.replace(";", "")
        word = word.replace(".", "")
        word = word.replace(",", "")
        word = word.replace('\'', "")
        word = word.replace('\"', "")
        #words below are ones where we want to connect together to ensure these words are joined
        word = word.replace("trumps", "trump")
        word = word.replace("russian", "russia")
        word = word.replace("russians", "russia")
        word = word.replace("dems", "democrat")
        word = word.replace("democrats", "democrat")
        word = word.replace("republicans", "republican")
        word = word.replace("libs", "liberal")
        word = word.replace("liberals", "liberal")
        word = word.replace("conservatives", "conservative")
        word = word.replace("taxes", "tax")
        word = word.replace("socialists", "socialist")
        word = word.replace("nationalists", "nationalist")
        word = word.replace("globalists", "globalist")
        word = word.replace("corrupted", "corrupt")
        newWords.append(word)
    return newWords

In [540]:
def collect(soup):
    articlesDF = []
    wordsDF = []
    articleList = soup.find('section', {'class': "collection collection-article-list has-load-more"})
    articles = articleList.findAll('article')
    print("articles length: ", len(articles))
    #TODO click on see more
    for article in articles:
        linkHTML = article.find('a')
        articleTime = article.find('span', {'class': 'time'}).text
        if 'minutes' in articleTime or 'minute' in articleTime:
            continue
        if 'hour' in articleTime or 'hours' in articleTime:
            if '9' in articleTime or '8' in articleTime or '7' in articleTime or '6' in articleTime or '5' in articleTime  or '4' in articleTime  or '3' in articleTime or '2' in articleTime or '1' in articleTime:
                continue
        link = linkHTML['href']
        if 'politics' in link:
            try:
                url = 'https://www.foxnews.com' + link
                driver.get(url)
                #print(driver.current_url)
                #htmlArticle = requests.get('https://www.foxnews.com/' + link)
                soupArticle = BeautifulSoup(driver.page_source, 'html.parser')
                #get article title

                #words in title
                header = soupArticle.find('header', {'class': 'article-header'})
                title = header.find('h1').text
                print(title)
                wordsInTitle = title.split()
                wordsInTitle = filterWords(wordsInTitle)

                getIcons = driver.find_element_by_class_name('article-social')
                getIcons = getIcons.find_element_by_class_name('social-icons')
                commentButton = getIcons.find_element_by_class_name('comments')
                #commentButton = commentButton.find_element_by_tag_name('a')

                #IT DOESNT CLICK
                try:
                    commentButton.click()
                    time.sleep(10)

                    iframe = driver.find_element_by_class_name('sppre_frame-container')
                    iframe = iframe.find_element_by_tag_name('iframe')
                    #iframe = driver.find_elements_by_id('bb4b0ab3b34f383d8da32a70bad542e1-iframe')
                    driver.switch_to.frame(iframe)
                    newSoupArticle = BeautifulSoup(driver.page_source, 'html.parser')
                    #wrong >:(
                    commentCount = newSoupArticle.find('span', {'class': 'param-messagesCount'}).text
                    commentCount = commentCount.replace(",", "")
                    #print(commentCount)
                    commentCount = int(commentCount)
                    articlesDF.append({'url': url, 'articleTitle': title,
                                    'commentCount': commentCount})
                    for word in wordsInTitle:

                        wordsDF.append({'url': url, 'articleTitle': title, 'word': word,
                                        'commentCount': commentCount})
                except:
                    print("CANNOT GET COMMENT COUNT")
            except:
                print("ERROR BEFORE GETTING TO COMMENTS")

    return [articlesDF, wordsDF]

In [670]:
driver.get('https://www.foxnews.com/politics')

for x in range(1, 50):
    try:
        loadMore = driver.find_element_by_class_name('load-more')
        loadMore.click()
    except:
        print("cant click load more")
    time.sleep(.2)
time.sleep(5)


soup = BeautifulSoup(driver.page_source, 'html.parser')
dfs = collect(soup)


NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=71.0.3578.80)
  (Driver info: chromedriver=2.43.600229 (3fae4d0cda5334b4f533bede5a4787f7b832d052),platform=Mac OS X 10.11.6 x86_64)


In [543]:
articlesDF = pd.DataFrame(dfs[0], columns = ['url', 'articleTitle', 'commentCount'])
wordsDF = pd.DataFrame(dfs[1], columns = ['url', 'articleTitle', 'word', 'commentCount'])

In [732]:
#removing words that have no politial connection (filler words) - scores are most likely coincidental
wordsDF = wordsDF[wordsDF.word != 'to']
wordsDF = wordsDF[wordsDF.word != 'in']
wordsDF = wordsDF[wordsDF.word != 'for']
wordsDF = wordsDF[wordsDF.word != 'of']
wordsDF = wordsDF[wordsDF.word != 'on']
wordsDF = wordsDF[wordsDF.word != 'a']
wordsDF = wordsDF[wordsDF.word != 'at']
wordsDF = wordsDF[wordsDF.word != 'be']
wordsDF = wordsDF[wordsDF.word != 'the']
wordsDF = wordsDF[wordsDF.word != 'with']
wordsDF = wordsDF[wordsDF.word != 'by']
wordsDF = wordsDF[wordsDF.word != 'from']
wordsDF = wordsDF[wordsDF.word != 'is']
wordsDF = wordsDF[wordsDF.word != 'about']
wordsDF = wordsDF[wordsDF.word != 'as']
wordsDF = wordsDF[wordsDF.word != 'over']
wordsDF = wordsDF[wordsDF.word != 'team']
wordsDF = wordsDF[wordsDF.word != 'out']
wordsDF = wordsDF[wordsDF.word != 'they']
wordsDF = wordsDF[wordsDF.word != 'that']
wordsDF = wordsDF[wordsDF.word != 'over']
wordsDF = wordsDF[wordsDF.word != 'end']
wordsDF = wordsDF[wordsDF.word != 'wants']
wordsDF = wordsDF[wordsDF.word != 'called']
wordsDF = wordsDF[wordsDF.word != 'calls']
wordsDF = wordsDF[wordsDF.word != 'over']
wordsDF = wordsDF[wordsDF.word != '$$']
wordsDF = wordsDF[wordsDF.word != 'foundation']
wordsDF = wordsDF[wordsDF.word != 'have']
wordsDF = wordsDF[wordsDF.word != '9th']
wordsDF = wordsDF[wordsDF.word != 'into']
wordsDF = wordsDF[wordsDF.word != 'after']
wordsDF = wordsDF[wordsDF.word != 'key']
wordsDF = wordsDF[wordsDF.word != 'among']
wordsDF = wordsDF[wordsDF.word != 'could']
wordsDF = wordsDF[wordsDF.word != 'says']
wordsDF = wordsDF[wordsDF.word != 'has']
wordsDF = wordsDF[wordsDF.word != 'him']

#it wont let me name it anything else
wordCounts = wordsDF['word'].value_counts()
wordCountsDF = pd.DataFrame(wordCounts, columns= ['word'])
wordCountsDF = wordCountsDF.reset_index()
wordCountsDF.columns = ['word', 'articleAppearanceCount']

#for word in words


In [662]:
averageCommentsPerWord = wordsDF.groupby('word', as_index=False)['commentCount'].mean()
only1 = wordCounts[wordCounts == 1]
for word in only1.index:
    averageCommentsPerWord = averageCommentsPerWord[averageCommentsPerWord.word != word]
        
averageCommentsPerWord
averageCommentsPerWord = averageCommentsPerWord.sort_values('commentCount', ascending=False)
articlesDF = articlesDF.sort_values('commentCount', ascending=False)

In [731]:
articlesDF.to_csv('foxArticlesCommentCount.csv', index = False)
wordCountsDF.to_csv('foxWordCounts.csv', index = False)
averageCommentsPerWord.to_csv('foxWordAvgCommentCount.csv', index = False)


In [724]:
wordCountAvgComment = []
for title in articlesDF['articleTitle']:
    commentCount = articlesDF.loc[articlesDF['articleTitle'] == title, 'commentCount'].iloc[0]
    wordsInTitle = title.split()
    wordCount = len(wordsInTitle)
    wordCountAvgComment.append({'wordCount': wordCount, 'commentCount': commentCount})

wordCountAvgCommentAllLengths = pd.DataFrame(wordCountAvgComment)


wordCountAvgComment = wordCountAvgCommentAllLengths.groupby('wordCount', as_index=False)['commentCount'].mean()

        

wordCountAvgComment['commentCount'] = round(wordCountAvgComment['commentCount'], 2)
wordCountAvgComment = wordCountAvgComment.sort_values('commentCount', ascending=False)

counts = []
for count in wordCountAvgComment['wordCount']:
    totalArticlesWithLength = 0
    for count2 in wordCountAvgCommentAllLengths['wordCount']:
        if count == count2:
            totalArticlesWithLength += 1
    counts.append({'wordCount':count, 'totalCount': totalArticlesWithLength})
    
counts = pd.DataFrame(counts)
wordCountAvgComment = pd.merge(counts, wordCountAvgComment, on='wordCount')
wordCountAvgComment.columns = ['totalArticlesWithTitleCount', 'wordCountInTitle', 'averageCommentCount']

In [730]:
wordCountAvgComment.to_csv('foxWordCountAvgCommentCount.csv', index = False)